In [1]:
import os
import sys
import pandas as pd
from IPython import display as ipd

import tensorflow as tf

tf.__version__

'2.4.1'

In [2]:
# test audio sample
test_data = '../data/test-samples/test_data.csv'

df = pd.read_csv(test_data)
print(df)


                     audio                            text
0  test_wav/070078fb60.wav      তেঁতুলিয়ার ভজনপুর গ্রামের
1  test_wav/070091fd89.wav          জবাবদিহিতায় আনার জন্য
2  test_wav/070a2946a9.wav      বিষয়টি পুলিশকে জানানো হয়
3  test_wav/071d32bd73.wav       ১৯৮৭ সালের ২০ ফেব্রুয়ারি
4  test_wav/072f9bb48f.wav     অফিস সুটের সর্বশেষ ভার্সনটি
5  test_wav/073598e69d.wav              ১০ গোলে পরাজিত করে
6  test_wav/0750033e3e.wav  সবচেয়ে দরিদ্র এলাকাগুলোর একটি
7  test_wav/0772cc6e7c.wav       আকাশের দিকেই ছুঁড়তে থাকে
8  test_wav/07738a801d.wav      চাঁদনী রাতের ভালোবাসার ফটো


In [3]:
# read sample audio and play it
wav_dir = '../data/test-samples/'
for i, d in df.iterrows():
    audio, text = os.path.join(wav_dir, d['audio']), d['text']
    print(f"text: {text}")
    ipd.display(ipd.Audio(audio))

text: তেঁতুলিয়ার ভজনপুর গ্রামের


text: জবাবদিহিতায় আনার জন্য


text: বিষয়টি পুলিশকে জানানো হয়


text: ১৯৮৭ সালের ২০ ফেব্রুয়ারি


text: অফিস সুটের সর্বশেষ ভার্সনটি


text: ১০ গোলে পরাজিত করে


text: সবচেয়ে দরিদ্র এলাকাগুলোর একটি


text: আকাশের দিকেই ছুঁড়তে থাকে


text: চাঁদনী রাতের ভালোবাসার ফটো


In [4]:
# read character sets
chars_db_path = '../db/chars_75.csv'

unique_chars = pd.read_csv(chars_db_path, skip_blank_lines=False)['chars']
unique_chars = unique_chars.to_list()

print(f"Total {len(unique_chars)} chars: {unique_chars}")

Total 71 chars: [' ', '।', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঋ', 'এ', 'ঐ', 'ও', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']


In [5]:
# add bnasr to sys path
sys.path.append(os.path.abspath('../'))

In [6]:
from bnasr import utils
from bnasr.utils import VectorizeChar
from bnasr.model import Transformer

In [7]:
max_target_len = 30
vectorizer = VectorizeChar(unique_chars, max_len=max_target_len)
vocab_size = len(vectorizer.get_vocabulary())

print(f"Vocab size: {vocab_size}")

Vocab size: 75


In [8]:

model = Transformer(
    num_hid=128,
    num_head=2,
    num_feed_forward=256,
    target_maxlen=max_target_len,
    num_layers_enc=2,
    num_layers_dec=1,
    num_classes= 77
)

In [9]:
# set checkpoint path
model_checkpoint = '../models'

# load checkpoint
latest = tf.train.latest_checkpoint(model_checkpoint)
model.load_weights(latest)

In [10]:
def predict(source, target_start_token_idx, target_maxlen):
        """Performs inference over one batch of inputs using greedy decoding."""
        bs = tf.shape(source)[0]
        enc = model.encoder(source)
        dec_input = tf.ones((bs, 1), dtype=tf.int32) * target_start_token_idx
        dec_logits = []
        for i in range(target_maxlen-1):
            dec_out = model.decode(enc, dec_input)
            logits = model.classifier(dec_out)
            logits = tf.argmax(logits, axis=-1, output_type=tf.int32)
            last_logit = tf.expand_dims(logits[:, -1], axis=-1)
            dec_logits.append(last_logit)
            dec_input = tf.concat([dec_input, last_logit], axis=-1)
        return dec_input

In [11]:
target_start_token_idx=2
target_end_token_idx = 3
idx_to_char = vectorizer.get_vocabulary()

for i, d in df.iterrows():
    # take audio and it's actual text
    audio, text = os.path.join(wav_dir, d['audio']), d['text']
    print(f"Sample [ {i+1} ] target text: {text}")
    print(f"Target audio: {audio}")
    ipd.display(ipd.Audio(audio))
    
    target_audio = utils.path_to_audio(audio)
    target_audio = tf.expand_dims(target_audio, 0)
    # perform prediction on given target audio file
    preds = predict(target_audio, target_start_token_idx, target_maxlen=max_target_len)
    
    preds = preds.numpy()[0]
    prediction = []
    for idx in preds:
        prediction.append(idx_to_char[idx])
        if idx == target_end_token_idx:
            break
    
    prediction = ''.join(prediction)
    print(f"prediction: {prediction}")
    print('-'*50)

Sample [ 1 ] target text: তেঁতুলিয়ার ভজনপুর গ্রামের
Target audio: ../data/test-samples/test_wav/070078fb60.wav


prediction: <তেঁতুলিয়ার ভজনপুর গ্রামের>
--------------------------------------------------
Sample [ 2 ] target text: জবাবদিহিতায় আনার জন্য
Target audio: ../data/test-samples/test_wav/070091fd89.wav


prediction: <জবাবদিহিতায় আনার জন্য>
--------------------------------------------------
Sample [ 3 ] target text: বিষয়টি পুলিশকে জানানো হয়
Target audio: ../data/test-samples/test_wav/070a2946a9.wav


prediction: <বিষয়টি পুলিশকে জানানো হয়>
--------------------------------------------------
Sample [ 4 ] target text: ১৯৮৭ সালের ২০ ফেব্রুয়ারি
Target audio: ../data/test-samples/test_wav/071d32bd73.wav


prediction: <১৯৮৭ সালের ২০ ফেব্রুয়ারি>
--------------------------------------------------
Sample [ 5 ] target text: অফিস সুটের সর্বশেষ ভার্সনটি
Target audio: ../data/test-samples/test_wav/072f9bb48f.wav


prediction: <অফিস সুটের সর্বশেষ ভার্সনটি>
--------------------------------------------------
Sample [ 6 ] target text: ১০ গোলে পরাজিত করে
Target audio: ../data/test-samples/test_wav/073598e69d.wav


prediction: <১০ গোলে পরাজিত করে>
--------------------------------------------------
Sample [ 7 ] target text: সবচেয়ে দরিদ্র এলাকাগুলোর একটি
Target audio: ../data/test-samples/test_wav/0750033e3e.wav


prediction: <সবচেয়ে দরিদ্র এলাকাগুলোর এক>
--------------------------------------------------
Sample [ 8 ] target text: আকাশের দিকেই ছুঁড়তে থাকে
Target audio: ../data/test-samples/test_wav/0772cc6e7c.wav


prediction: <আকাশের দিকেই ছুঁড়তে থাকে>
--------------------------------------------------
Sample [ 9 ] target text: চাঁদনী রাতের ভালোবাসার ফটো
Target audio: ../data/test-samples/test_wav/07738a801d.wav


prediction: <চাঁদনী রাতের ভালোবাসার ফটো>
--------------------------------------------------
